# ADAPT-VQE

ADAPT-VQE was introduced by [Grimsley et al](https://www.nature.com/articles/s41467-019-10988-2) as a solution to the often impracticably deep, and not necessarily accurate, static VQE circuits. In ansatze like UCCSD, one easily reaches the order of thousands of gates, even for modestly sized molecules. In UCCSD and its generalized UCCGSD, the number of fermionic excitations in the ansatz scales like $\mathcal{O}(N^2M^2)$, and $\mathcal{O}(N^4)$ respectively. Here $N$ refers to the number of spin-orbitals in the problem basis, and $M$ the number of electrons.

In ADAPT-VQE, an ansatz which approximates not UCCSD/UCCGSD, but in fact FCI, is built iteratively. Over a series of cycles, the ansatz circuit is grown to achieve an approximation to FCI with a minimal number of circuit elements. In this way, ADAPT-VQE is a meta-VQE. At each step, a new ansatz is defined, and its parameters optimized according to conventional VQE. As the cycles proceed, the ansatz grows in both complexity and expressibility. This algorithm comes at the expense of a significant increase in measurement overhead. In order to identify the best operator to append to the present ansatz circuit, a large number of measurements are performed to rank the available operators in order of their ability to further reduce the ansatz state energy.

In this notebook, we'll explore the implementation of this algorithm, by using the implementation available in qSDK. 

## Fermionic-Inspired Qubit-ADAPT
We are going to use the unitary generalized singles and doubles to establish a set of pool generators to use in our implementation of ADAPT-VQE. 

In [ ]:
import numpy as np

from qsdk.toolboxes.ansatz_generator._general_unitary_cc import uccgsd_generator as uccgsd_pool
from qsdk.toolboxes.ansatz_generator._paired_unitary_cc import get_generalized_singles, get_paired_doubles
from qsdk.toolboxes.operators import FermionOperator

def get_pool(n_spin_orbitals):
    """Use the number of spin orbitals and the qsdk uccgsd_generator function that returns a list
    of FermionOperators to sample from in order to choose the best operator to add to the ansatz.
    This is the default pool used in the ansatz
    Args:
        n_spin_orbitals (int): number of spin_orbitals in the Hamiltonian
    
    Returns:
        pool_generators (list of FermionOperator): list of generators
    """

    pool_generators = uccgsd_pool(n_spin_orbitals)
    # n_orbs=n_spin_orbitals//2
    # all_terms = get_generalized_singles(n_orbs) + get_paired_doubles(n_orbs)
    # pool_generators = list()
    # for item in all_terms:
    #    pool_generators.append(FermionOperator(item[0], 1))
    return pool_generators

## Adaptive-Ansatz
Now that we have a pool of FermionOperators that we can send into `AdaptSolver` class to run Adapt-VQE.

In [ ]:
from qsdk.electronic_structure_solvers import ADAPTSolver

## ADAPT-VQE In Practice
We are going to consider H$_4$ as a linear chain. We begin by instantiating the molecule object from pyscf's `gto.Mole` class, and feed this into the `ADAPTSolver` method we imported above.

In [ ]:
from pyscf import gto
H4 = [("H", (0, 0, 0)), ("H", (0, 0, 1)), ("H", (0, 0, 2)), ("H", (0, 0, 3))]
mol = gto.Mole()
mol.atom = H4
mol.basis = "sto-3g"
mol.build()

opt_dict = {"molecule": mol, "frozen_orbitals": 0, "pool": get_pool, "tol": 0.01, "max_cycles": 6, "verbose": False}

adapt_solver = ADAPTSolver(opt_dict)
adapt_solver.build()
adapt_solver.simulate()

After 6 cycles, we force the cycle to terminate. We can now compare the results against the predictions of FCI.

In [ ]:
import matplotlib.pyplot as plt
from qsdk.toolboxes.molecular_computation.molecular_data import MolecularData
from qsdk.electronic_structure_solvers.fci_solver import FCISolver

solver = FCISolver()
exact = solver.simulate(mol)
errors = np.array(adapt_solver.energies) - exact
fig,ax = plt.subplots(1,1)
ax.plot(errors)
ax.set_xlabel('ADAPT Iteration')
ax.set_ylabel('Error (Ha)')
ax.set_title('ADAPT-vqe: H$_4$')
ax.set_yscale('log')
print(f'Final Error: {errors[-1] :.4E}')

Ok so after 15 cycles, we have an error of 0.14 mHa, within chemical accuracy of FCI. How does this all compare against UCCSD-VQE?

In [ ]:
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver

vqe_uccsd = VQESolver({'molecule': mol})
vqe_uccsd.build()
vqe_uccsd.simulate()

In [ ]:
print(f'ADAPT-VQE ERROR: {adapt_solver.energies[-1] - exact :0.4E} Ha')
print(f'UCCSD-VQE ERROR: {vqe_uccsd.optimal_energy - exact :0.4E} Ha')

From the perspective of energy accuracy, the two have reached very similar results, within a factor of two. The big advantage here however, are in the resources required for this ansatz circuit:

In [ ]:
print(f'ADAPT RESOURCES:\n {adapt_solver.get_resources()}\n')
print(f'UCCSD RESOURCES:\n {vqe_uccsd.get_resources()}')

We have managed here to reduce the total number of gates by about 30%, the number of 2-qubit gates by a factor of nearly 50%, and variational gates by about 50%. 

We note that ADAPT-VQE has run with 8 fewer variational parameters than UCCSD-VQE. This  reduces the classical overhead which is important when one wishes to look at larger molecules. With Adapt-VQE, the scaling should be better than the $\mathcal{O}(N^2M^2)$ scaling of UCCSD-VQE.

## Conclusion

In this notebook, we've explored a simple implementation of the ADAPT-VQE algorithm, using tools available through the qSDK. We have demonstrated how we can leverage these tools to reduce the effort involved in prototyping new quantum algorithms. This allows the user to focus exclusively on the specific components relevant to their research objectives, without getting bogged down in re-building standard infrastructure from scratch. 

The ADAPT-VQE algorithm provides the flexibility to explore different choices of generator pools. Feel free to try out the many options available for the fermionic system you are examining. If you wish to look at a QubitOperator Hamiltonian, please look at the qubit_adapt.ipynb notebook in the user_notebooks folder.
